## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Layer Convolutional:
* [torch.nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)(in_channels, out_channels, kernel_size, stride=1, padding=0)

Layere Pooling:
* [torch.nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)(kernel_size, stride=None, padding=0)
*  [torch.nn.AveragePool2d](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html)(kernel_size, stride=None, padding=0)

Layere Adaptive Pool, intalnit adesea si ca Global Pool:
* [torch.nn.AdaptiveAvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html)(output_size)
* [torch.nn.AdaptiveMaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveMaxPool2d.html)(output_size)

Layer de liniarizare:

* [torch.nn.Flatten()](https://pytorch.org/docs/stable/generated/torch.flatten.html)



Four **hyperparameters** control the size of the output volume:
* **Depth**: number of filters, as each filter _looks_ at different areas of the input:
* **Stride**: the step taken when _sliding_ the filter. (Usually 1 or 2, 3 - uncommon).
* **Zero-Padding**: size of the number of 0s that surround the border of the input volume. Example: If you want to the same width and height for input and output.
* **Dilation**: Distance between elements of the convolutional kernel.



**Why Pooling Layer?**

1. Modifica volumul de input (input volume) in reprezentari _mai mici_ si mai usor de _manevrat_.
2. Opereaza independent pe fiecare Activation Map.

<img src="https://computersciencewiki.org/images/9/9e/MaxpoolSample.png" width="425" height="300"/> <img src="https://miro.medium.com/v2/resize:fit:517/0*lIcR0gOMK0xzTr6_.png" width="425" height="300"/> 






**Why Adaptive Pooling Layer?**

1. Folosite de regula in etapele finale de constructie a unei arhitecturi de tipul _ConvNet_ pentru a inlocui fully-connected layers.
2. Incearca sa previna *overfitting phenomenon* fortand feature maps sa retina informatia **globala** care este relevanta pentru task-ul acestei _ConvNet_ (clasificare, identifcare etc.)

<img src="https://www.researchgate.net/publication/338079465/figure/fig4/AS:905983672987648@1593014748012/The-difference-of-max-pooling-and-global-max-pooling.ppm" width="725" height="300"/>


<img src="https://drive.google.com/uc?id=11l7Xsh-iQmASvXTkgH2MgtA01XCW6CAC">


[Visualise them Here](https://github.com/vdumoulin/conv_arithmetic).

In [ ]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizate pentru a reduce dimensiunea la una prestabilita, util cand marimea input ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 49, 49])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 25, 25] # Stride & Padding
*   [1, 10, 32, 32]
*  [1, 3, 2, 2]



In [3]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=10, kernel_size=2, stride=2),
    nn.MaxPool2d(2),
)
print("Conv1 ",layer(dummy_input_tensor).shape)

layer = nn.Sequential(
    nn.Conv2d(3, 10, kernel_size=5, stride=3, padding=0)
)
print("Conv2 ",layer(dummy_input_tensor).shape)

layer = nn.Sequential(
    nn.MaxPool2d(2),
    nn.MaxPool2d(2),
    nn.MaxPool2d(2),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=2, padding=0), #remove 4
)
print("Conv3 ",layer(dummy_input_tensor).shape)

Conv1  torch.Size([1, 10, 25, 25])
Conv2  torch.Size([1, 10, 32, 32])
Conv3  torch.Size([1, 3, 2, 2])


## Instantierea seturilor de date

In [1]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __collate_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *collate_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize


def collate_fn(examples):
  processed_images = []
  processed_labels = []

  for example in examples:
    tensor_image = to_tensor(example[0])
    normalized_tensor_image = normalize(tensor_image, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    vector_image = normalized_tensor_image.unsqueeze(0)
    processed_images.append(vector_image)
    
    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)

  return torch_images, torch_labels


train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii de 2 ori (0.25 p).
    * Liniarizati iesirea din cel de-al doilea strat convolutional (0.25 p).
    * Adaugat stratul final de tipul `fully-connected` (0.25 p).
    * Folositi o functie de activare la alegere (Exemplu [RELU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)) (0.25 p).

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [36]:
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1),
      nn.ReLU(),
    )

    self.linear = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=8*8*128, out_features=10),
      nn.Softmax(dim=-1),
    )

  def forward(self,x):
    x = self.conv(x)
    x = self.linear(x)
    return x

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Numarul de epoci (0.25 p)
  * Retea (0.25 p)
  * Optimizator (0.25 p)
  * Alegeti functia de cost (0.25 p)

In [37]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 20

# Definiti reteaua
network = Net()

# Definiti optimizatorul
optimizer = optim.SGD(network.parameters(),lr=1e-4, momentum=0.9)
"""
Dupa definirea optimizatorului si dupa fiecare iteratie de antrenare, trebuie 
apelata functia zero_grad() pentru a seta valoare tuturor gradientilor la zero.
"""
# Completati aici codul pentru seta valoare tuturor gradientilor la zero
optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
# https://neptune.ai/blog/pytorch-loss-functions
loss_fn = nn.CrossEntropyLoss()

## Definirea functiei de antrenare

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def test_acc(net: nn.Module, test_loader: DataLoader):
  net.eval()

  total = 0
  correct = 0

  for test_images, test_labels in test_loader:
    test_images = test_images.to(device)
    test_labels = test_labels.to(device)
    total += len(test_images)
    out_class = torch.argmax(net(test_images))
    correct += torch.sum(out_class == test_labels)

  return correct / total * 100


def train_fn(epochs: int, train_loader: DataLoader, test_loader: DataLoader, 
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  
  net = net.to(device)
  # Iteram prin numarul de epoci
  for e in range(epochs):
    net.train()

    # Iteram prin fiecare batch din dataloader
    for images, labels in train_loader:
      images = images.to(device)
      labels = labels.to(device)
      # Aplicam reteaua neurala pe imaginile din batch-ul curent
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe etichetele imaginilor 
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a actualiza parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()
    
    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Calculam acuratetea
    acc = test_acc(net, test_loader)
    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e + 1, acc))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [41]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.302001476287842
Acuratetea la finalul epocii 1 este 12.32%
Loss-ul la finalul epocii 1 are valoarea 2.3018457889556885
Acuratetea la finalul epocii 2 este 12.96%
Loss-ul la finalul epocii 2 are valoarea 2.302072048187256
Acuratetea la finalul epocii 3 este 13.42%
Loss-ul la finalul epocii 3 are valoarea 2.3017592430114746
Acuratetea la finalul epocii 4 este 13.81%
Loss-ul la finalul epocii 4 are valoarea 2.3011281490325928
Acuratetea la finalul epocii 5 este 14.04%
Loss-ul la finalul epocii 5 are valoarea 2.3017261028289795
Acuratetea la finalul epocii 6 este 14.53%
Loss-ul la finalul epocii 6 are valoarea 2.301527261734009
Acuratetea la finalul epocii 7 este 14.86%
Loss-ul la finalul epocii 7 are valoarea 2.3011884689331055
Acuratetea la finalul epocii 8 este 15.00%
Loss-ul la finalul epocii 8 are valoarea 2.3011200428009033
Acuratetea la finalul epocii 9 este 15.38%
Loss-ul la finalul epocii 9 are valoarea 2.300820827484131
Acuratetea la fin

## Reteaua LeNet

### Cerinte
  * **(3p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o.



![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet

_Question:_ Care este diferenta dintre `tanh` si `softmax`? De ce credeti ca peste ultimul layer (cel de output) a fost aplicata functia `softmax`?


In [50]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    self.conv = nn.Sequential(
        nn.Conv2d(3, 6, 5, 1),
        nn.Tanh(),
        nn.AvgPool2d(2),
        nn.Conv2d(6, 16, 5, 1),
        nn.Tanh(),
        nn.AvgPool2d(2),
        nn.Conv2d(16, 120, 5, 1),
        nn.Tanh()
    )

    self.linear = nn.Sequential(
        nn.Flatten(),
        nn.Linear(120, 84),
        nn.Tanh(),
        nn.Linear(84, 10),
        nn.Softmax(dim=-1)
    )

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    x = self.conv(x)
    x = self.linear(x)
    return x

## Optional: Reteaua AlexNet ❤️

❗Daca alegeti aceasta retea veti continua sa rezolvati exercitiile urmatoare pentru reteaua AlexNet.

Pentru a usura volumul de munca si obtine o retea AlexNet comparabila in dificultate cu LeNet, urmati acesti pasi:

✔️ Includeti functii de activare intre layere (exemplu ReLU).

✔️ Va folositi doar de prima subsectiune din schema figurii arhitecturii AlexNet (adica doar Conv1 si Conv2 blocks).

✔️ Inputul vostru se opreste la un minimum size de 8x8.

✔️ Modificati output-ul retelei sa prezica 10 clase in loc de 1000 de clase.





![alt text](https://anhreynolds.com/img/alexnet.png)

Figura arhitectura AlexNet.

![alt text](https://anhreynolds.com/img/alexnet-parameters.png)

Tabel arhitectura AlexNet


In [ ]:
import torch.nn as nn

class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    self.conv = nn.Sequential(
        nn.Conv2d(3, 96, 11, 4),
        nn.MaxPool2d(3, 2),
        nn.Conv2d(96, 256, 3, 2),
        nn.MaxPool2d(3, 2),
        nn.Conv2d(256, 384, 3, 1, 1),
        nn.Conv2d(384, 384, 3, 1),
        nn.Conv2d(384, 256, 3, 1),
        nn.MaxPool2d(3, 2),
    )
    self.linear = nn.Sequential(
        nn.Flatten(),
        nn.Linear(9216, 4096),
        nn.Linear(4096, 10),
        nn.Softmax(dim=-1)
    )

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    return x

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * Redefiniti obiectele pentru a antrena reteaua LeNet

In [51]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 20

# Definiti reteaua
lenet = LeNet()

# Definiti optimizatorul
lenet_optimizer = optim.SGD(lenet.parameters(), lr=1e-4, momentum=0.9)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Antrenarea retelei LeNet

In [52]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.3025283813476562
Acuratetea la finalul epocii 1 este 10.24%
Loss-ul la finalul epocii 1 are valoarea 2.301919460296631
Acuratetea la finalul epocii 2 este 10.24%
Loss-ul la finalul epocii 2 are valoarea 2.302401304244995
Acuratetea la finalul epocii 3 este 10.24%
Loss-ul la finalul epocii 3 are valoarea 2.302412748336792
Acuratetea la finalul epocii 4 este 10.23%
Loss-ul la finalul epocii 4 are valoarea 2.3028252124786377
Acuratetea la finalul epocii 5 este 10.23%
Loss-ul la finalul epocii 5 are valoarea 2.3024821281433105
Acuratetea la finalul epocii 6 este 10.23%
Loss-ul la finalul epocii 6 are valoarea 2.302541971206665
Acuratetea la finalul epocii 7 este 10.24%
Loss-ul la finalul epocii 7 are valoarea 2.3023200035095215
Acuratetea la finalul epocii 8 este 10.25%
Loss-ul la finalul epocii 8 are valoarea 2.302184820175171
Acuratetea la finalul epocii 9 este 10.26%
Loss-ul la finalul epocii 9 are valoarea 2.30242919921875
Acuratetea la finalu

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* [torch.nn.BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) (num_features)
* [torch.nn.InstanceNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.InstanceNorm2d.html) (num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de alte functii de activare:

* ReLU
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu aceste elemente in cadrul retelei LeNet definita mai devreme, pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.

**Punctaj:** 0.6p / configuratie.

0.6p din care:
- 0.4p modificarea retelei
- 0.1p obtinerea rezultatelor
- 0.1p afisarea acestora si explicatie.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  


In [57]:
import torch.nn as nn

class LeNet1(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(3, 6, 5, 1),
        nn.BatchNorm2d(6),
        nn.Tanh(),
        nn.AvgPool2d(2),
        nn.Conv2d(6, 16, 5, 1),
        nn.BatchNorm2d(16),
        nn.Tanh(),
        nn.AvgPool2d(2),
        nn.Conv2d(16, 120, 5, 1),
        nn.BatchNorm2d(120),
        nn.Tanh()
    )

    self.linear = nn.Sequential(
        nn.Flatten(),
        nn.Linear(120, 84),
        nn.BatchNorm1d(84),
        nn.Tanh(),
        nn.Linear(84, 10),
        nn.Softmax(dim=-1)
    )

  def forward(self,x):
    x = self.conv(x)
    x = self.linear(x)
    return x

In [58]:
epochs = 20

lenet = LeNet1()
lenet_optimizer = optim.SGD(lenet.parameters(), lr=1e-4, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.306962728500366
Acuratetea la finalul epocii 1 este 8.16%
Loss-ul la finalul epocii 1 are valoarea 2.3053653240203857
Acuratetea la finalul epocii 2 este 9.26%
Loss-ul la finalul epocii 2 are valoarea 2.3011231422424316
Acuratetea la finalul epocii 3 este 10.53%
Loss-ul la finalul epocii 3 are valoarea 2.3010377883911133
Acuratetea la finalul epocii 4 este 11.91%
Loss-ul la finalul epocii 4 are valoarea 2.2992970943450928
Acuratetea la finalul epocii 5 este 13.37%
Loss-ul la finalul epocii 5 are valoarea 2.2964699268341064
Acuratetea la finalul epocii 6 este 14.78%
Loss-ul la finalul epocii 6 are valoarea 2.3003618717193604
Acuratetea la finalul epocii 7 este 15.85%
Loss-ul la finalul epocii 7 are valoarea 2.2935078144073486
Acuratetea la finalul epocii 8 este 16.94%
Loss-ul la finalul epocii 8 are valoarea 2.2930352687835693
Acuratetea la finalul epocii 9 este 17.97%
Loss-ul la finalul epocii 9 are valoarea 2.2899279594421387
Acuratetea la fi

In [59]:
import torch.nn as nn

class LeNet2(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(3, 6, 5, 1),
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.AvgPool2d(2),
        nn.Conv2d(6, 16, 5, 1),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.AvgPool2d(2),
        nn.Conv2d(16, 120, 5, 1),
        nn.BatchNorm2d(120),
        nn.ReLU(),
    )

    self.linear = nn.Sequential(
        nn.Flatten(),
        nn.Linear(120, 84),
        nn.BatchNorm1d(84),
        nn.ReLU(),
        nn.Linear(84, 10),
        nn.Softmax(dim=-1)
    )

  def forward(self,x):
    x = self.conv(x)
    x = self.linear(x)
    return x

In [60]:
epochs = 20

lenet = LeNet2()
lenet_optimizer = optim.SGD(lenet.parameters(), lr=1e-4, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.3023681640625
Acuratetea la finalul epocii 1 este 10.27%
Loss-ul la finalul epocii 1 are valoarea 2.298398494720459
Acuratetea la finalul epocii 2 este 10.84%
Loss-ul la finalul epocii 2 are valoarea 2.300441265106201
Acuratetea la finalul epocii 3 este 11.28%
Loss-ul la finalul epocii 3 are valoarea 2.2991750240325928
Acuratetea la finalul epocii 4 este 11.89%
Loss-ul la finalul epocii 4 are valoarea 2.299428701400757
Acuratetea la finalul epocii 5 este 12.30%
Loss-ul la finalul epocii 5 are valoarea 2.2968640327453613
Acuratetea la finalul epocii 6 este 12.79%
Loss-ul la finalul epocii 6 are valoarea 2.2950003147125244
Acuratetea la finalul epocii 7 este 12.99%
Loss-ul la finalul epocii 7 are valoarea 2.2999870777130127
Acuratetea la finalul epocii 8 este 13.32%
Loss-ul la finalul epocii 8 are valoarea 2.2979230880737305
Acuratetea la finalul epocii 9 este 13.57%
Loss-ul la finalul epocii 9 are valoarea 2.299198627471924
Acuratetea la finalu

In [61]:
import torch.nn as nn

class LeNet3(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(3, 6, 5, 1, bias=False),
        nn.BatchNorm2d(6),
        nn.GELU(),
        nn.AvgPool2d(2),
        nn.Conv2d(6, 16, 5, 1, bias=False),
        nn.BatchNorm2d(16),
        nn.GELU(),
        nn.AvgPool2d(2),
        nn.Conv2d(16, 120, 5, 1, bias=False),
        nn.BatchNorm2d(120),
        nn.GELU(),
    )

    self.linear = nn.Sequential(
        nn.Flatten(),
        nn.Linear(120, 84, bias=False),
        nn.BatchNorm1d(84),
        nn.GELU(),
        nn.Linear(84, 10),
        nn.Softmax(dim=-1)
    )

  def forward(self,x):
    x = self.conv(x)
    x = self.linear(x)
    return x

In [62]:
epochs = 20

lenet = LeNet3()
lenet_optimizer = optim.SGD(lenet.parameters(), lr=1e-4, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.30248761177063
Acuratetea la finalul epocii 1 este 10.14%
Loss-ul la finalul epocii 1 are valoarea 2.300225257873535
Acuratetea la finalul epocii 2 este 10.54%
Loss-ul la finalul epocii 2 are valoarea 2.2965474128723145
Acuratetea la finalul epocii 3 este 11.02%
Loss-ul la finalul epocii 3 are valoarea 2.3016161918640137
Acuratetea la finalul epocii 4 este 11.50%
Loss-ul la finalul epocii 4 are valoarea 2.298255681991577
Acuratetea la finalul epocii 5 este 11.86%
Loss-ul la finalul epocii 5 are valoarea 2.2988061904907227
Acuratetea la finalul epocii 6 este 12.54%
Loss-ul la finalul epocii 6 are valoarea 2.296520471572876
Acuratetea la finalul epocii 7 este 12.96%
Loss-ul la finalul epocii 7 are valoarea 2.294252634048462
Acuratetea la finalul epocii 8 este 13.47%
Loss-ul la finalul epocii 8 are valoarea 2.297171115875244
Acuratetea la finalul epocii 9 este 14.01%
Loss-ul la finalul epocii 9 are valoarea 2.290009021759033
Acuratetea la finalul